In [19]:
import pandas as pd

X_train= pd.read_csv('X_train_fill_v0.csv', index_col=0)
Y_train= pd.read_csv('Y_train.csv',index_col=0)
X_test= pd.read_csv('X_test_fill_v0.csv',index_col=0)

X_train.head()

,patient_id,cohort,sexM,gene,age_at_diagnosis,age,ledd,time_since_intake_on,time_since_intake_off,on,off
Index,,,,,,,,,,,
0,3332,0,0,1,48.5,52.1,607.0,1.9,14.779591,7.0,38.620296
1,3332,0,0,1,48.5,53.0,666.0,1.9,17.600000,12.0,44.000000
2,3332,0,0,1,48.5,53.9,717.0,1.2,14.878310,6.0,39.662327
3,3332,0,0,1,48.5,54.8,770.0,1.5,14.727922,11.0,40.183342
4,3332,0,0,1,48.5,56.9,885.0,0.3,14.326217,24.0,41.399048


Crée un dataset ou des qu'une valuer est decroissante on la suprimer (n'a pas booster les resulats)
Crer un dataset ou on utilise la somme cumulitavie pour creer des off croissantes (degrade les résultats)
Ajouter une feature 0,1 pour conisder si c'est un bonne feture (proche de la valeur precedente), améluire un peu les résultats
Estimer target-off plutot que target directement, ne boost pas. 
pas de patient en commun 
(log puis exp pour fill data degrade, cliping améliore un peu)
mettre 3 points minimum pour ridge ne change rien, 4 améliore legerement (5 degrdation)


In [41]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error

# --- Load and Merge Data ---
X_train = pd.read_csv('X_train_fill_v0.csv', index_col=0)
Y_train = pd.read_csv('Y_train.csv', index_col=0)
data_all = X_train.copy()
data_all['target'] = Y_train['target']

# --- Define Patient-Level Split Function (to avoid leakage) ---
def split_by_patient(df):
    unique_patients = df['patient_id'].unique()
    np.random.seed(42)
    np.random.shuffle(unique_patients)
    split_idx = int(0.8 * len(unique_patients))
    train_patients = unique_patients[:split_idx]
    valid_patients = unique_patients[split_idx:]
    train_data = df[df['patient_id'].isin(train_patients)]
    valid_data = df[df['patient_id'].isin(valid_patients)]
    return train_data, valid_data

# Split the merged data into train and validation sets
train_set, val_set = split_by_patient(data_all)

# --- Train Baseline Model on the Train Set ---
# We'll use all features except the target as predictors.
features = [col for col in train_set.columns if col != 'target']
baseline_model =  RandomForestRegressor(n_estimators=100, random_state=42)
baseline_model.fit(train_set[features], train_set['target'])

# --- Predict on the Validation Set with the Baseline Model ---
val_set = val_set.copy()  # ensure we don't modify original val_set
val_set['baseline_pred'] = baseline_model.predict(val_set[features])

# --- Per-Patient Calibration on the Validation Set ---
# For each patient in the validation set with at least 3 rows,
# we perform a leave-one-out calibration using Ridge regression with age as predictor.
# For patients with fewer than 3 rows, we keep the baseline predictions.

# We'll create a Series to store the final predictions (aligned with val_set's index)
final_pred = pd.Series(index=val_set.index, dtype=float)

# Group the validation set by patient_id
for patient, group in val_set.groupby('patient_id'):
    if len(group) >= 3:
        # For each measurement of the patient, perform leave-one-out prediction
        patient_preds = []
        indices = group.index
        for i in range(len(group)):
            # Use all rows except the i-th one as training for the patient-specific model
            train_grp = group.drop(index=indices[i])
            test_grp = group.loc[[indices[i]]]
            # Use 'age' as the predictor; fit Ridge using actual target values
            ridge_model = Ridge(alpha=1.0)
            ridge_model.fit(train_grp[['age']], train_grp['target'])
            pred = ridge_model.predict(test_grp[['age']])[0]
            patient_preds.append(pred)
        # Assign the calibrated predictions to the corresponding indices
        final_pred.loc[indices] = patient_preds
    else:
        # For patients with fewer than 3 measurements, use the baseline prediction
        final_pred.loc[group.index] = group['baseline_pred']

# --- Evaluate the Final Predictions ---
mse_final = mean_squared_error(val_set['target'], final_pred)
print(f"Final MSE on the validation set after per-patient Ridge calibration: {mse_final:.4f}")

Final MSE on the validation set after per-patient Ridge calibration: 6.0840


In [17]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# --- Load and Merge Data ---
X_train = pd.read_csv('X_train_fill_v0.csv', index_col=0)
Y_train = pd.read_csv('Y_train.csv', index_col=0)
data = X_train.copy()
data['target'] = Y_train['target']

# Base features: all columns except target.
base_features = [col for col in data.columns if col != 'target']

# --- Define Patient-Level Split Function (to avoid leakage) ---
def split_by_patient(df):
    unique_patients = df['patient_id'].unique()
    np.random.seed(42)
    np.random.shuffle(unique_patients)
    split_idx = int(0.8 * len(unique_patients))
    train_patients = unique_patients[:split_idx]
    valid_patients = unique_patients[split_idx:]
    train_data = df[df['patient_id'].isin(train_patients)]
    valid_data = df[df['patient_id'].isin(valid_patients)]
    return train_data, valid_data

# --- Define Model Evaluation Function ---
def evaluate_model(df, feature_cols):
    # Split the data by patient
    train_data, valid_data = split_by_patient(df)
    X_train_model = train_data[feature_cols]
    y_train_model = train_data['target']
    X_valid_model = valid_data[feature_cols]
    y_valid_model = valid_data['target']
    
    # Train a simple Linear Regression model
    model = LinearRegression()
    model.fit(X_train_model, y_train_model)
    
    # Predict on validation set and compute MSE
    y_pred = model.predict(X_valid_model)
    mse = mean_squared_error(y_valid_model, y_pred)
    return mse

# --- Evaluate Baseline Model ---
baseline_mse = evaluate_model(data, base_features)
print(f"Baseline MSE (using original features): {baseline_mse:.4f}")

# --- Define Candidate Feature Engineering Functions ---
# Each candidate is defined as a lambda or function that takes the full dataframe and returns a pd.Series.
candidates = {}

# Interaction Features
candidates['on_off_ratio'] = lambda df: (df['on'] / (df['off'] + 1)).fillna(0)

# Time-Derived Features
def off_cumavg(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].transform(lambda x: x.expanding().mean())
candidates['off_cumavg'] = off_cumavg

def on_cumavg(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['on'].transform(lambda x: x.expanding().mean())
candidates['on_cumavg'] = on_cumavg

# Lag Features
def off_lag1(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].shift(1).fillna(method='bfill')
candidates['off_lag1'] = off_lag1

def on_lag1(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['on'].shift(1).fillna(method='bfill')
candidates['on_lag1'] = on_lag1

# Exponential Weighted Moving Average (EWMA) of 'off'
def off_ewma(df, span=2):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].transform(lambda x: x.ewm(span=span, adjust=False).mean())
candidates['off_ewma'] = off_ewma

# Acceleration: second difference of 'off'
def off_acceleration(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].diff().diff().fillna(0)
candidates['off_acceleration'] = off_acceleration


# --- Cumulative Feature Evaluation ---
cumulative_features = base_features.copy()  # start with baseline features
results = {}

# Evaluate baseline (already computed)
results['baseline'] = baseline_mse
print("\nCumulative Feature Evaluation:")
print(f"Baseline: MSE = {baseline_mse:.4f}")

# Iterate over candidates in the defined order, stacking each new feature
for feat_name, func in candidates.items():
    # Add the candidate feature to the dataset
    data[feat_name] = func(data)
    # Append the candidate feature to our cumulative feature list
    cumulative_features.append(feat_name)
    
    mse_candidate = evaluate_model(data, cumulative_features)
    results[feat_name] = mse_candidate
    print(f"After adding '{feat_name}': MSE = {mse_candidate:.4f} (Delta: {baseline_mse - mse_candidate:+.4f})")
    
# Optionally, display a summary of all results.
print("\nSummary of Cumulative Feature Evaluation:")
for feat, mse in results.items():
    print(f"{feat:20s}: MSE = {mse:.4f}")

Baseline MSE (using original features): 59.8034

Cumulative Feature Evaluation:
Baseline: MSE = 59.8034
After adding 'on_off_ratio': MSE = 59.3925 (Delta: +0.4109)
After adding 'off_cumavg': MSE = 45.3145 (Delta: +14.4889)
After adding 'on_cumavg': MSE = 45.1795 (Delta: +14.6239)
After adding 'off_lag1': MSE = 44.8686 (Delta: +14.9348)


/var/folders/zn/74t7wv5n5r593c2rckpn_pbr0000gn/T/ipykernel_35257/2612472309.py:70: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df_sorted.groupby('patient_id')['off'].shift(1).fillna(method='bfill')
/var/folders/zn/74t7wv5n5r593c2rckpn_pbr0000gn/T/ipykernel_35257/2612472309.py:75: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df_sorted.groupby('patient_id')['on'].shift(1).fillna(method='bfill')


After adding 'on_lag1': MSE = 44.6184 (Delta: +15.1850)
After adding 'off_ewma': MSE = 43.8456 (Delta: +15.9578)
After adding 'off_acceleration': MSE = 43.8352 (Delta: +15.9682)

Summary of Cumulative Feature Evaluation:
baseline            : MSE = 59.8034
on_off_ratio        : MSE = 59.3925
off_cumavg          : MSE = 45.3145
on_cumavg           : MSE = 45.1795
off_lag1            : MSE = 44.8686
on_lag1             : MSE = 44.6184
off_ewma            : MSE = 43.8456
off_acceleration    : MSE = 43.8352


In [4]:
data.head()

,patient_id,cohort,sexM,gene,age_at_diagnosis,age,ledd,time_since_intake_on,time_since_intake_off,on,off,target,on_off_ratio,off_cumavg,off_lag1
Index,,,,,,,,,,,,,,,
12,1,1,1,2,57.7,58.7,415.07993,1.828529,13.824179,16.056835,18.00000,22.5,0.845097,18.000000,18.0
13,1,1,1,2,57.7,60.1,460.00000,2.005669,14.081960,17.000000,28.00000,26.6,0.586207,23.000000,18.0
14,1,1,1,2,57.7,61.4,522.00000,1.758086,13.768008,17.000000,16.00000,29.3,1.000000,20.666667,28.0
15,1,1,1,2,57.7,62.9,604.00000,1.747771,13.810826,19.000000,19.24389,30.7,0.938555,20.310972,16.0
55261,2,0,0,2,55.0,56.5,525.59000,1.786182,13.681935,9.968306,4.00000,5.5,1.993661,4.000000,4.0


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# --- Load and Merge Data ---
X_train = pd.read_csv('X_train_fill_v0.csv', index_col=0)
Y_train = pd.read_csv('Y_train.csv', index_col=0)
data = X_train.copy()
data['target'] = Y_train['target']

# Base features: all columns except target.
base_features = [col for col in data.columns if col != 'target']

# --- Define Candidate Feature Engineering Functions ---
# Each candidate is defined as a lambda or function that takes the full dataframe and returns a pd.Series.
candidates = {}

# Interaction Features
candidates['on_off_ratio'] = lambda df: (df['on'] / (df['off'] + 1)).fillna(0)

# Time-Derived Features
def off_cumavg(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].transform(lambda x: x.expanding().mean())
candidates['off_cumavg'] = off_cumavg

def on_cumavg(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['on'].transform(lambda x: x.expanding().mean())
candidates['on_cumavg'] = on_cumavg

# Lag Features
def off_lag1(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].shift(1).fillna(method='bfill')
candidates['off_lag1'] = off_lag1

def on_lag1(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['on'].shift(1).fillna(method='bfill')
candidates['on_lag1'] = on_lag1

# Exponential Weighted Moving Average (EWMA) of 'off'
def off_ewma(df, span=2):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].transform(lambda x: x.ewm(span=span, adjust=False).mean())
candidates['off_ewma'] = off_ewma

# Acceleration: second difference of 'off'
def off_acceleration(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].diff().diff().fillna(0)
candidates['off_acceleration'] = off_acceleration

/var/folders/zn/74t7wv5n5r593c2rckpn_pbr0000gn/T/ipykernel_35257/2129508841.py:31: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df_sorted.groupby('patient_id')['off'].shift(1).fillna(method='bfill')
/var/folders/zn/74t7wv5n5r593c2rckpn_pbr0000gn/T/ipykernel_35257/2129508841.py:37: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df_sorted.groupby('patient_id')['on'].shift(1).fillna(method='bfill')


,patient_id,cohort,sexM,gene,age_at_diagnosis,age,ledd,time_since_intake_on,time_since_intake_off,on,off,target,off_cumavg,on_cumavg,off_lag1,on_lag1,off_ewma
Index,,,,,,,,,,,,,,,,,
0,3332,0,0,1,48.5,52.1,607.00000,1.900000,14.779859,7.0,38.620296,34.7,38.620296,7.000000,22.000000,11.170789,38.620296
1,3332,0,0,1,48.5,53.0,666.00000,1.900000,17.600000,12.0,44.000000,38.1,41.310148,9.500000,38.620296,7.000000,42.206765
2,3332,0,0,1,48.5,53.9,717.00000,1.200000,14.878010,6.0,39.662327,41.6,40.760874,8.333333,44.000000,12.000000,40.510473
3,3332,0,0,1,48.5,54.8,770.00000,1.500000,14.727335,11.0,40.183342,44.9,40.616491,9.000000,39.662327,6.000000,40.292386
4,3332,0,0,1,48.5,56.9,885.00000,0.300000,14.325004,24.0,41.399048,52.0,40.773003,12.000000,40.183342,11.000000,41.030161
5,3332,0,0,1,48.5,57.5,842.43866,2.400000,14.612772,16.0,41.746390,53.9,40.935234,12.666667,41.399048,24.000000,41.507647
6,3332,0,0,1,48.5,58.9,835.00000,2.641195,14.697359,10.0,37.000000,57.8,40.373058,12.285714,41.746390,16.000000,38.502549
7,3332,0,0,1,48.5,59.9,888.00000,0.200000,9.600000,31.0,36.000000,60.1,39.826425,14.625000,37.000000,10.000000,36.834183
8,3332,0,0,1,48.5,60.9,952.00000,1.500000,14.882829,11.0,43.714672,62.0,40.258453,14.222222,36.000000,31.000000,41.421176


In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

#########################
# Feature Engineering Functions
#########################
def off_cumavg(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].transform(lambda x: x.expanding().mean())

def on_cumavg(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['on'].transform(lambda x: x.expanding().mean())

def off_lag1(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].shift(1).fillna(method='bfill')

def on_lag1(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['on'].shift(1).fillna(method='bfill')

def off_ewma(df, span=2):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].transform(lambda x: x.ewm(span=span, adjust=False).mean())

#########################
# Load & Process Training Data
#########################
X_train = pd.read_csv('X_train_fill_v0.csv', index_col=0)
Y_train = pd.read_csv('Y_train.csv', index_col=0)
train_data = X_train.copy()
train_data['target'] = Y_train['target']

# Apply feature engineering on training data
train_data['off_cumavg'] = off_cumavg(train_data)
train_data['on_cumavg']  = on_cumavg(train_data)
train_data['off_lag1']   = off_lag1(train_data)
train_data['on_lag1']    = on_lag1(train_data)
train_data['off_ewma']   = off_ewma(train_data, span=2)

# Define features (all columns except target)
features = [col for col in train_data.columns if col != 'target']

# Train baseline model using CatBoostRegressor
baseline_model = CatBoostRegressor(random_state=42, verbose=0)
baseline_model.fit(train_data[features], train_data['target'])

#########################
# Load & Process Test Data
#########################
X_test = pd.read_csv('X_test_fill_v0.csv', index_col=0)
# Apply same feature engineering on test data
X_test['off_cumavg'] = off_cumavg(X_test)
X_test['on_cumavg']  = on_cumavg(X_test)
X_test['off_lag1']   = off_lag1(X_test)
X_test['on_lag1']    = on_lag1(X_test)
X_test['off_ewma']   = off_ewma(X_test, span=2)

# Predict baseline values for X_test using the baseline model
X_test = X_test.copy()  # ensure we don't modify the original X_test
X_test['baseline_pred'] = baseline_model.predict(X_test[features])

#########################
# Per-Patient Calibration on Test Data
#########################
# For patients with at least 3 measurements, we recalibrate the first 3 rows using leave-one-out Ridge regression.
# Since we don't have the true target in test data, we use the baseline predictions as pseudo-targets for calibration.
final_pred = pd.Series(index=X_test.index, dtype=float)

# Process each patient separately
for patient, group in X_test.groupby('patient_id'):
    group = group.sort_values('age')
    if len(group) >= 3:
        # Select the first three measurements (as requested)
        selected_indices = group.index[:3]
        for idx in selected_indices:
            # Leave-one-out: train on all other rows for this patient using "age" to predict baseline_pred
            train_grp = group.drop(index=idx)
            test_grp  = group.loc[[idx]]
            ridge_model = Ridge(alpha=1.0)
            ridge_model.fit(train_grp[['age']], train_grp['baseline_pred'])
            pred = ridge_model.predict(test_grp[['age']])[0]
            final_pred.loc[idx] = pred
        # For any additional measurements, use the baseline prediction
        remaining_indices = set(group.index) - set(selected_indices)
        final_pred.loc[list(remaining_indices)] = group.loc[list(remaining_indices), 'baseline_pred']
    else:
        # For patients with fewer than 3 measurements, use the baseline prediction directly.
        final_pred.loc[group.index] = group['baseline_pred']

#########################
# Create Submission CSV
#########################
output = final_pred.reset_index()
output.columns = ['Index', 'target']
output.index = np.arange(len(output))  # Reset index from 0

# Save the output to a CSV file for Kaggle submission.
output.to_csv('test_predictions.csv', index=False)
print("Predictions saved to 'test_predictions.csv'")

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

###########################
# 1. Load & Process Training Data
###########################
X_train = pd.read_csv('X_train_fill_v0.csv', index_col=0)
Y_train = pd.read_csv('Y_train.csv', index_col=0)
train_data = X_train.copy()
train_data['target'] = Y_train['target']

# --- Feature Engineering Functions ---
def off_cumavg(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].transform(lambda x: x.expanding().mean())

def on_cumavg(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['on'].transform(lambda x: x.expanding().mean())

def off_lag1(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].shift(1).fillna(method='bfill')

def on_lag1(df):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['on'].shift(1).fillna(method='bfill')

def off_ewma(df, span=2):
    df_sorted = df.sort_values('age')
    return df_sorted.groupby('patient_id')['off'].transform(lambda x: x.ewm(span=span, adjust=False).mean())

# Apply feature engineering on training data
train_data['off_cumavg'] = off_cumavg(train_data)
train_data['on_cumavg']  = on_cumavg(train_data)
train_data['off_lag1']   = off_lag1(train_data)
train_data['on_lag1']    = on_lag1(train_data)
train_data['off_ewma']   = off_ewma(train_data, span=2)

# Define features (all columns except 'target')
features = [col for col in train_data.columns if col != 'target']

# --- Train Baseline Model (CatBoostRegressor) ---
baseline_model = CatBoostRegressor(random_state=42, verbose=0)
baseline_model.fit(train_data[features], train_data['target'])

###########################
# 2. Load & Process Test Data
###########################
X_test = pd.read_csv('X_test_fill_v0.csv', index_col=0)
test_data = X_test.copy()  # test set does not have a target column

# Apply the same feature engineering on test data
test_data['off_cumavg'] = off_cumavg(test_data)
test_data['on_cumavg']  = on_cumavg(test_data)
test_data['off_lag1']   = off_lag1(test_data)
test_data['on_lag1']    = on_lag1(test_data)
test_data['off_ewma']   = off_ewma(test_data, span=2)

# --- Predict Baseline Target on Test Data ---
test_data['baseline_pred'] = baseline_model.predict(test_data[features])

###########################
# 3. Per-Patient Calibration on Test Data
###########################
# We'll recalibrate the first 3 measurements per patient using leave-one-out Ridge regression (using only "age").
# For patients with fewer than 3 measurements, we simply keep the baseline predictions.
final_pred = pd.Series(index=test_data.index, dtype=float)

# Group test data by patient_id and sort by age within each group
for patient, group in test_data.groupby('patient_id'):
    group = group.sort_values('age')
    if len(group) >= 3:
        # Use the first 3 measurements
        selected_indices = group.index[:3]
        for idx in selected_indices:
            # Leave-one-out: train Ridge on all rows except the current one using "age" to predict baseline_pred
            train_grp = group.drop(index=idx)
            test_grp  = group.loc[[idx]]
            ridge_model = Ridge(alpha=1.0)
            ridge_model.fit(train_grp[['age']], train_grp['baseline_pred'])
            pred = ridge_model.predict(test_grp[['age']])[0]
            final_pred.loc[idx] = pred
        # For the remaining measurements, keep the baseline prediction
        remaining_indices = set(group.index) - set(selected_indices)
        final_pred.loc[list(remaining_indices)] = group.loc[list(remaining_indices), 'baseline_pred']
    else:
        # For patients with fewer than 3 measurements, use the baseline prediction directly.
        final_pred.loc[group.index] = group['baseline_pred']



/var/folders/zn/74t7wv5n5r593c2rckpn_pbr0000gn/T/ipykernel_35257/964354825.py:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df_sorted.groupby('patient_id')['off'].shift(1).fillna(method='bfill')
/var/folders/zn/74t7wv5n5r593c2rckpn_pbr0000gn/T/ipykernel_35257/964354825.py:30: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df_sorted.groupby('patient_id')['on'].shift(1).fillna(method='bfill')
/var/folders/zn/74t7wv5n5r593c2rckpn_pbr0000gn/T/ipykernel_35257/964354825.py:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return df_sorted.groupby('patient_id')['off'].shift(1).fillna(method='bfill')
/var/folders/zn/74t7wv5n5r593c2rckpn_pbr0000gn/T/ipykernel_35257/964354825.py:30: FutureWarning: Series.fillna with 

Predictions saved to 'test_predictions.csv'


In [51]:
# Create a submission DataFrame with a new Index starting at 0
submission = pd.DataFrame({'target': final_pred.values})
submission.insert(0, 'Index', range(len(final_pred)))

# Save to CSV
submission.to_csv('test_predictions.csv', index=False)
print("Predictions saved to 'test_predictions.csv'")

Predictions saved to 'test_predictions.csv'
